In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps

from ProjectOneCRUDModule import AnimalShelter


###########################
# Data Manipulation / Model
###########################
shelter = AnimalShelter()

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}, {"_id": 0}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png' # customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Joey Paul Eli Haynes, February 2023'))),
    html.Hr(),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
            alt='customer image', style={'width' : '150px', 'height' : '150px'})),
    
    html.Div(className='row',
        style={'display' : 'block'},
        children=[dcc.Dropdown(options=[{'label': 'Water Rescue', 'value' : 'Water Rescue'}, 
                                        {'label': 'Mountain Rescue', 'value' : 'Mountain Rescue'}, 
                                        {'label': 'Disaster Rescue', 'value' : 'Disaster Rescue'},
                                        {'label': 'All', 'value' : 'All'}], value='All', id='filter-type'),
        ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=5 
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            dcc.Graph(id='graph-id'),
            className='col s12 m6'
            ),
        html.Div(
            id='map-id',
            className='col s12 m6'
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################

    
#This callback enables button functionality to sort by filter options
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
        if filter_type == 'Water Rescue':
            waterRescueQuery = {"animal_type":"Dog", 
                                   "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", 
                                                   "Newfoundland"]}, 
                                   "sex_upon_outcome":"Intact Female", 
                                   "age_upon_outcome_in_weeks":{"$gte":26}, 
                                   "age_upon_outcome_in_weeks":{"$lte":156}}
            df = pd.DataFrame(list(shelter.read(waterRescueQuery, {"_id": 0})))
        elif filter_type == 'Mountain Rescue':
            mountainRescueQuery = {"animal_type":"Dog", 
                                   "breed":{"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                                                   "Siberian Husky", "Rottweiler"]}, 
                                   "sex_upon_outcome":"Intact Male", 
                                   "age_upon_outcome_in_weeks":{"$gte":26}, 
                                   "age_upon_outcome_in_weeks":{"$lte":156}}
            df = pd.DataFrame(list(shelter.read(mountainRescueQuery, {"_id": 0})))
        elif filter_type == 'Disaster Rescue':
            disasterRescueQuery = {"animal_type":"Dog", 
                                   "breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                                                   "Bloodhound", "Rottweiler"]}, 
                                   "sex_upon_outcome":"Intact Male", 
                                   "age_upon_outcome_in_weeks":{"$gte":20}, 
                                   "age_upon_outcome_in_weeks":{"$lte":300}}
            df = pd.DataFrame(list(shelter.read(disasterRescueQuery, {"_id": 0})))
        else:
            df = pd.DataFrame.from_records(shelter.read({}, {"_id": 0}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        return (data, columns)

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#This callback implements the pie chart
@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dffPie = pd.DataFrame.from_dict(viewData)     
    fig = px.pie(dffPie, names="breed")
    fig.update_layout(width=400, height=300)
    
    return fig

#This callback implements the geolocation chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at[30.75, -97.48]
    return [
        dl.Map(style={'width':'500px', 'height':'250px'},center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            #Marker with tool tip and popup
            dl.Marker(position=[30.75, -97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app